In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [7]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.55, 0.9)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312      1.0 -1.739972e-06                  NaN           NaN
0.000312      NaN           NaN                  1.0  3.126841e-05
0.000750      2.0 -1.691339e-06                  2.0  9.040280e-05
0.001052      3.0 -1.547010e-06                  3.0  9.648426e-06
0.001476      4.0 -1.321156e-06                  4.0  6.108845e-06
0.002070      5.0 -9.952471e-07                  5.0  4.615716e-06
0.002904      6.0 -5.475467e-07                  6.0  3.389450e-06
0.004074      7.0  4.767115e-08                  7.0  2.660213e-06
0.005714      8.0  8.226801e-07                  8.0  2.243122e-06
0.008015      9.0  1.812960e-06                  9.0  2.142128e-06
0.011243     10.0  3.060598e-06                 10.0  2.240994e-06
0.015771     11.0  4.614836e-06                 11.0  2.607051e-06
0.022122     12.0  6.531551e-06                 12.0  3.157286e-06
0.031031     13.0  8.864591e-06                 13.0  3.003058e-06
0.043528     14.0  1.133795e-05                 14.0  3.556406e-06
0.061057     15.0  1.354673e-05                 15.0  5.245778e-06
0.085645     16.0  1.568813e-05                 16.0  6.584991e-06
0.120136     17.0  1.805783e-05                 17.0  7.600377e-06
0.168516     18.0  2.068564e-05                 18.0  8.692708e-06
0.236378     19.0  2.359606e-05                 19.0  9.860489e-06
0.331549     20.0  2.681014e-05                 20.0  1.104773e-05
0.465100     21.0  2.998143e-05                 21.0  1.169705e-05
0.652400     22.0  2.748034e-05                 22.0  1.090293e-05
0.915100     23.0  1.889242e-05                 23.0  7.869713e-06
1.283650     24.0  1.125698e-05                 24.0  4.679436e-06
1.800600     25.0  6.080012e-06                 25.0  2.380913e-06
2.525700     26.0  2.675685e-06                 26.0  8.636971e-07
3.542800     27.0  5.348150e-07                 27.0 -8.787447e-08
4.969550     28.0 -6.137896e-07                 28.0 -6.112121e-07
6.970850     29.0 -1.139138e-06                 29.0 -8.711562e-07
9.778100     30.0 -1.427074e-06                 30.0 -1.017121e-06
13.715850    31.0 -1.611923e-06                 31.0 -1.114504e-06
19.239350    32.0 -1.729567e-06                 32.0 -1.178494e-06
26.987250    33.0 -1.759404e-06                 33.0 -1.206051e-06
37.855300    34.0 -1.731227e-06                 34.0 -1.196213e-06
53.100050    35.0 -1.681306e-06                 35.0 -1.231762e-06
73.887500    36.0 -1.619160e-06                 36.0 -1.272437e-06
97.662500    37.0 -1.569147e-06                 37.0 -1.284456e-06
121.437500   38.0 -1.577798e-06                 38.0 -1.312655e-06
145.212500   39.0 -2.219998e-06                 39.0 -1.833773e-06
168.987500   40.0 -2.966301e-06                 40.0 -2.435667e-06
192.762500   41.0 -3.871196e-06                 41.0 -3.170343e-06
216.537500   42.0 -4.634976e-06                 42.0 -3.851945e-06
240.312500   43.0 -5.426818e-06                 43.0 -4.594204e-06
264.087500   44.0 -6.373958e-06                 44.0 -5.461047e-06
287.862500   45.0 -7.432478e-06                 45.0 -6.418758e-06
311.637500   46.0 -8.190233e-06                 46.0 -7.183249e-06
335.412500   47.0 -7.924950e-06                 47.0 -7.143506e-06
359.187500   48.0 -6.067689e-06                 48.0 -6.030576e-06
382.962500   49.0 -2.715752e-06                 49.0 -4.376614e-06
406.737500   50.0  7.779120e-06                 50.0  2.107950e-06
430.512500   51.0  3.238814e-05                 51.0  1.886778e-05
454.287500   52.0  7.902770e-05                 52.0  5.234602e-05
478.062500   53.0  1.564385e-04                 53.0  1.104513e-04
501.837500   54.0  2.732544e-04                 54.0  2.016675e-04
525.612500   55.0  4.367586e-04                 55.0  3.295621e-04
549.387500   56.0  6.511740e-04                 56.0  5.02087

# Fluxes by Layer

CRD                              CLIRAD            \
                         flug          fldg      fnetg       flug      fldg   
pressure     level                                                            
0.000000e+00 1     -15.040846  0.000000e+00 -15.040846        NaN       NaN   
1.000000e-08 1            NaN           NaN        NaN -15.043313  0.000002   
6.244000e-04 2     -15.040846  1.095316e-10 -15.040846 -15.043313  0.000002   
8.759000e-04 3     -15.040846  1.579624e-10 -15.040846 -15.043313  0.000002   
1.228600e-03 4     -15.040846  2.398500e-10 -15.040846 -15.043313  0.000002   
1.723400e-03 5     -15.040846  3.783968e-10 -15.040846 -15.043313  0.000002   
2.417400e-03 6     -15.040846  6.122651e-10 -15.040846 -15.043313  0.000002   
3.390900e-03 7     -15.040846  1.006281e-09 -15.040846 -15.043313  0.000002   
4.756500e-03 8     -15.040846  1.668696e-09 -15.040846 -15.043313  0.000002   
6.672000e-03 9     -15.040846  2.779945e-09 -15.040846 -15.043313  0.000002   
9.358900e-03 10    -15.040846  4.640309e-09 -15.040846 -15.043313  0.000002   
1.312780e-02 11    -15.040846  7.748377e-09 -15.040846 -15.043313  0.000002   
1.841450e-02 12    -15.040846  1.293049e-08 -15.040846 -15.043313  0.000002   
2.583020e-02 13    -15.040846  2.155316e-08 -15.040846 -15.043313  0.000002   
3.623230e-02 14    -15.040846  3.586128e-08 -15.040846 -15.043313  0.000002   
5.082340e-02 15    -15.040846  5.910685e-08 -15.040846 -15.043313  0.000002   
7.129060e-02 16    -15.040846  9.558640e-08 -15.040846 -15.043313  0.000002   
1.000000e-01 17    -15.040846  1.519189e-07 -15.040846 -15.043313  0.000002   
1.402710e-01 18    -15.040846  2.388141e-07 -15.040846 -15.043313  0.000002   
1.967600e-01 19    -15.040846  3.727633e-07 -15.040845 -15.043313  0.000002   
2.759970e-01 20    -15.040846  5.791010e-07 -15.040845 -15.043313  0.000002   
3.871000e-01 21    -15.040846  8.965925e-07 -15.040845 -15.043313  0.000002   
5.431000e-01 22    -15.040846  1.380935e-06 -15.040844 -15.043313  0.000002   
7.617000e-01 23    -15.040846  2.017127e-06 -15.040844 -15.043313  0.000003   
1.068500e+00 24    -15.040846  2.700195e-06 -15.040843 -15.043313  0.000003   
1.498800e+00 25    -15.040846  3.383049e-06 -15.040842 -15.043313  0.000003   
2.102400e+00 26    -15.040846  4.060060e-06 -15.040842 -15.043314  0.000004   
2.949000e+00 27    -15.040846  4.725633e-06 -15.040842 -15.043314  0.000004   
4.136600e+00 28    -15.040847  5.376167e-06 -15.040842 -15.043314  0.000004   
5.802500e+00 29    -15.040848  6.039746e-06 -15.040842 -15.043315  0.000005   
8.139200e+00 30    -15.040849  6.781209e-06 -15.040842 -15.043315  0.000005   
1.141700e+01 31    -15.040850  7.650383e-06 -15.040843 -15.043316  0.000006   
1.601470e+01 32    -15.040852  8.685508e-06 -15.040843 -15.043317  0.000006   
2.246400e+01 33    -15.040855  9.945181e-06 -15.040845 -15.043319  0.000007   
3.151050e+01 34    -15.040858  1.164865e-05 -15.040847 -15.043321  0.000008   
4.420010e+01 35    -15.040863  1.421099e-05 -15.040849 -15.043325  0.000009   
6.200000e+01 36    -15.040871  1.815119e-05 -15.040853 -15.043330  0.000012   
8.577500e+01 37    -15.040881  2.391443e-05 -15.040857 -15.043337  0.000016   
1.095500e+02 38    -15.040892  3.007958e-05 -15.040862 -15.043345  0.000020   
1.333250e+02 39    -15.040903  3.646769e-05 -15.040866 -15.043353  0.000024   
1.571000e+02 40    -15.040918  4.534817e-05 -15.040872 -15.043364  0.000030   
1.808750e+02 41    -15.040938  5.706903e-05 -15.040881 -15.043379  0.000038   
2.046500e+02 42    -15.040964  7.216508e-05 -15.040892 -15.043398  0.000049   
2.284250e+02 43    -15.040995  9.001272e-05 -15.040905 -15.043422  0.000062   
2.522000e+02 44    -15.041031  1.106585e-04 -15.040920 -15.043450  0.000077   
2.759750e+02 45    -15.041073  1.346414e-04 -15.040938 -15.043484  0.000095   
2.997500e+02 46    -15.041122  1.628027e-04 -15.040959 -15.043523  0.000116   
3.235250e+02 47    -15.041180  1.983866e-04 -15.040982 -15.043571  0.000144 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')